In [ ]:
import pandas as pd
import numpy as np
import re
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef

import shap

import pickle

## 1- IMPORT DATA

In [ ]:
#On recupère le dataframe mergé (voir kernel Kaggle)
df = pd.read_feather("data_feather.pkl")

## 2- TRAIN TEST SPLIT

In [ ]:
# On recupère les données avec une target
train = df[df["TARGET"].notnull()]

X_train = train.drop(["TARGET", "SK_ID_CURR"], axis = 1)
y_train = train["TARGET"]

In [ ]:
#On recupère le nombre de valeurs nulles
X_train_na = X_train.isna().sum()

#On recupère les variables avec plus de 90% de valeur non nulles
X_train_na_60 = list(X_train_na[(X_train_na*100/X_train.shape[0]) > 10].index)

#On supprime les variables ayant plus de 60% de valeurs nulles
X_train.drop(columns=X_train_na_60, inplace=True)

In [ ]:
%%time

#On remplace les valeurs nan restantes par 0
X_train.replace({np.nan : 0}, inplace= True)

In [ ]:
%%time

#On remplace les valeurs inf par la valeur max et les valeurs -inf par la valeur min
for c in X_train.columns:
    if X_train[c].isin([-np.inf, np.inf]).any():
        print(c)
        max_value = max(list(X_train[~(X_train[c].isin([-np.inf, np.inf, np.nan]))][c]))
        min_value = min(list(X_train[~(X_train[c].isin([-np.inf, np.inf, np.nan]))][c]))
    
        X_train[c].replace({np.inf : max_value, -np.inf : min_value}, inplace= True)

## 3- Gestion du déséquilibre des données (SMOTE)

In [ ]:
#On affiche la distribution de nos données cible (on observe un déséquilibre)
taux_0 = Counter(y_train)[0]*100/len(y_train)
taux_1 = Counter(y_train)[1]*100/len(y_train)

plt.bar(["Taux de 0 (négatif) : " + str(round(taux_0)) + "%"
         ,"Taux de 1 (positif) : " + str(round(taux_1))+ "%"]
        , [taux_0,taux_1])

In [ ]:
#On affiche le nombre de clients par classe
Counter(y_train)

In [ ]:
%%time
# Choix de la taille du nouveau dataset 
count_0 = Counter(y_train)[0]
distribution_of_samples = {0:count_0 , 1:50000}

# Sur-Echantillonnage en utilisant la méthode SMOTE
smote = SMOTE(sampling_strategy = distribution_of_samples, random_state=666)
X_under_tr, y_under_tr = smote.fit_resample(X_train, y_train)

In [ ]:
%%time
# Choix de la taille du nouveau dataset 
distribution_of_samples = {0:50000, 1:50000}

# Sous-Echantillonnage en utilisant la méthode NearMiss
smote = NearMiss(sampling_strategy = distribution_of_samples)
X_over_tr, y_over_tr = smote.fit_resample(X_under_tr, y_under_tr)

In [ ]:
#On affiche le nouveau nombre de clients par classe
Counter(y_over_tr)

In [ ]:
#On affiche la distribution de nos données cible (on observe un déséquilibre)
taux_0 = Counter(y_over_tr)[0]*100/len(y_over_tr)
taux_1 = Counter(y_over_tr)[1]*100/len(y_over_tr)

plt.bar(["Taux de 0 (négatif) : " + str(round(taux_0,1)) + "%"
         ,"Taux de 1 (positif) : " + str(round(taux_1,1))+ "%"]
        , [taux_0,taux_1])

In [ ]:
#On effectue un train test pour entrainer et tester notre modèle
X_tr, X_val, y_tr, y_val = train_test_split(X_over_tr, y_over_tr, stratify=y_over_tr
                                                                , train_size=0.8, random_state=666)

## 4- XGBoost avec SMOTE

In [ ]:
%%time

cls = xgb.XGBClassifier(use_label_encoder=False)
#                         , scale_pos_weight=Counter(y_over_tr)[0]/Counter(y_over_tr)[1])
# cls.set_params()
# scaler = StandardScaler()
# X_tr_scale = scaler.fit_transform(X_over_tr)
cls.fit(X_tr, y_tr)

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.model_selection import StratifiedKFold

In [ ]:
# params = {'n_estimators': [100, 150]}

In [ ]:
# %%time
# folds = 3
# param_comb = 2

# skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

# random_search = RandomizedSearchCV(cls, param_distributions=params, n_iter=param_comb
#                                    , scoring='f1', n_jobs=4, cv=skf.split(X_over_tr, y_over_tr)
#                                    , verbose=3, random_state=1001)

# random_search.fit(X_over_tr, y_over_tr)

In [ ]:
# random_search.best_estimator_

In [ ]:
#On affiche l'accuracy des données test
y_pred_xgb = cls.predict_proba(X_val)[:, 1]
[fpr_xgb, tpr_xgb, thr_xgb] = roc_curve(y_val, y_pred_xgb)

auc_xgb = auc(fpr_xgb, tpr_xgb)
print("Accuracy XGBoost : ", auc_xgb)

In [ ]:
#On affiche l'accuracy des données train (surapprentissage)
y_pred_xgb_tr = cls.predict_proba(X_tr)[:, 1]
[fpr_xgb_tr, tpr_xgb_tr, thr_xgb_tr] = roc_curve(y_tr, y_pred_xgb_tr)

auc_xgb_tr = auc(fpr_xgb_tr, tpr_xgb_tr)
print("Accuracy XGBoost Xtrain : ", auc_xgb_tr)

In [ ]:
#On affiche notre courbe ROC
plt.plot(fpr_xgb, tpr_xgb, color='coral', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1 - specificite', fontsize=14)
plt.ylabel('Sensibilite', fontsize=14)

## 5- Matrice de confusion et indicateurs clés

In [ ]:
#On compte les valeurs réelles
print("Valeurs réelles : ", Counter(y_val))

#On fixe 0.5 comme seuil par défaut
seuil = 0.5
y_pred_xgb_0_1 = [0 if x < seuil else 1 for x in y_pred_xgb]
print("Valeurs prédites : ", Counter(y_pred_xgb_0_1))

In [ ]:
#On affiche nos fonctions coût
print("Accuracy : ", round(accuracy_score(y_val, y_pred_xgb_0_1), 2))
print("Precision : ",round(precision_score(y_val, y_pred_xgb_0_1), 2))
print("Sensibility : ", round(recall_score(y_val, y_pred_xgb_0_1), 2))
print("F1 Score : ",round(f1_score(y_val, y_pred_xgb_0_1), 2))
print("MCC : ",round(matthews_corrcoef(y_val, y_pred_xgb_0_1), 2))


In [ ]:
#On calcule l'ensemble de nos fonctions coût pour différents seuil allant de 0 à 1
precision_tab = []
sensibility_tab = []
f1_score_tab = []
matthews_corrcoef_tab = []

rang = np.linspace(0.01,1,5)

for seuil in rang:
    y_pred_xgb_0_1 = []
    y_pred_xgb_0_1 = [0 if x < seuil else 1 for x in y_pred_xgb]
    precision_tab.append(round(precision_score(y_val, y_pred_xgb_0_1), 2))
    sensibility_tab.append(round(recall_score(y_val, y_pred_xgb_0_1), 2))
    f1_score_tab.append(round(f1_score(y_val, y_pred_xgb_0_1), 2))
    matthews_corrcoef_tab.append(round(matthews_corrcoef(y_val, y_pred_xgb_0_1), 2))

In [ ]:
#On affiche les valeurs de nos fonctions coût pour différents seuil allant de 0 à 1
fig = plt.figure(figsize=(15,5))

ax = fig.add_subplot(1,2,1)
plt.plot(rang, precision_tab, color='purple', label='Precision')
plt.plot(rang, sensibility_tab, color='green', label='Sensibility ')
ax.legend()

ax = fig.add_subplot(1,2,2)
plt.plot(rang, f1_score_tab, color='blue', label='F1 Score')
plt.plot(rang, matthews_corrcoef_tab, color='orange', label='MCC')
ax.legend()

In [ ]:
#On compte les valeurs réelles
print("Valeurs réelles : ", Counter(y_val))

#On fixe le seuil à 0.3 (le plus intéressant pour notre problème)
seuil = 0.8
y_pred_xgb_0_1 = [0 if x < seuil else 1 for x in y_pred_xgb]
print("Valeurs prédites : ", Counter(y_pred_xgb_0_1))

In [ ]:
print("Accuracy : ", round(accuracy_score(y_val, y_pred_xgb_0_1), 2))
print("Precision : ",round(precision_score(y_val, y_pred_xgb_0_1), 2))
print("Sensibility : ", round(recall_score(y_val, y_pred_xgb_0_1), 2))
print("F1 Score : ",round(f1_score(y_val, y_pred_xgb_0_1), 2))
print("MCC : ",round(matthews_corrcoef(y_val, y_pred_xgb_0_1), 2))

In [ ]:
pd.crosstab(y_val, np.array(y_pred_xgb_0_1), rownames=['True']
            , colnames=['Predicted'], margins=True)

In [ ]:
pd.crosstab(y_val, np.array(y_pred_xgb_0_1), rownames=['True']
            , colnames=['Predicted'], margins=True).apply(lambda r: round(r/y_val.count()*100,1), axis=1)

- La précision est la probabilité que le test soit positif si la client est un bon client
- La sensibilité est la probabilité que le test soit positif si la client est un mauvais client

In [ ]:
#On sauvegarde notre modèle au format pickle
with open("best_model_XGBoost_pickle.pkl", 'wb') as file:  
    pickle.dump(cls, file)

## 6- Comprehension du modèle

In [ ]:
#On importe le modèle shap pour l'interprétabilité de notre modèle
shap.initjs()

explainer = shap.TreeExplainer(cls)
shap_values = explainer.shap_values(X_over_tr)

In [ ]:
#On trace l'importance des variables de notre modèle
fig = shap.summary_plot(shap_values, X_over_tr, plot_type="bar")

In [ ]:
#On affiche l'ensemble des clients en fonction de l'importance de nos variables
shap.summary_plot(shap_values, X_over_tr)

In [ ]:
#On trace le waterfall d'un client eligible
shap.waterfall_plot(shap.Explanation(values=shap_values[2,:],
                                     base_values=explainer.expected_value,
                                     data=X_over_tr.iloc[2,:],
                                     feature_names=X_over_tr.columns.tolist()),max_display=10)

## 7- Création du dataframe pour le dashboard interactif

In [ ]:
#On prépare le dataframe n'ayant pas de target
test = df[df["TARGET"].isnull()]
X_test = test.drop(["TARGET"], axis = 1)

del df

In [ ]:
#On supprime les variables ayant plus de 10% de valeurs nulles
X_test.drop(columns=X_train_na_60, inplace=True)

In [ ]:
%%time

#On remplace les valeurs nan restantes par 0
X_test.replace({np.nan : 0}, inplace= True)

In [ ]:
%%time

#On remplace les valeurs inf par la valeur max et les valeurs -inf par la valeur min
for c in X_test.columns:
    if X_test[c].isin([-np.inf, np.inf]).any():
        print(c)
        max_value = max(list(X_test[~(X_test[c].isin([-np.inf, np.inf, np.nan]))][c]))
        min_value = min(list(X_test[~(X_test[c].isin([-np.inf, np.inf, np.nan]))][c]))
    
        X_test[c].replace({np.inf : max_value, -np.inf : min_value}, inplace= True)

In [ ]:
#On recupère le score et la valeur binaire definissant si le client est eligible ou non
target = cls.predict(X_test.iloc[:,1:])
score = [round(i,2) for i in cls.predict_proba(X_test.iloc[:,1:])[:, 1]]

In [ ]:
#On affiche le nombre de clients par classe
Counter(target)

In [ ]:
#On ajoute la target et le score à notre dataframe
X_test['SCORE'] = score
X_test['TARGET'] = target
X_test.reset_index(drop=True, inplace=True)

In [ ]:
#On sauvegarde le dataframe au format pickle
with open("data_customers.pkl", 'wb') as file:  
    pickle.dump(X_test, file)